    
----------
# CONTENTS

-    [Specify Input Data ](#0)  

-    [1. MRC analysis](#1)       
      
-    [2. MRC - Statistical analysis ](#2) 
            
-    [3. EPE analysis ](#3)


In [7]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
import matplotlib as mpl

In [8]:
from ipynb.fs.full.Whole_project_classes import *
from ipynb.fs.full.visualization_helper_functions import *

<a name='0'></a>
## Specify Input Data 

In [9]:
## path info
stripped_data_folder = 'data/stripped_data'
original_data_folder = 'data/ordered_data'
situations = ['controls', 'patients']
output_root_folder = 'output'
MRI_info_excel_name = 'MRI_informations.xlsx'
MRC_output_folder = 'MRC_results'   ## this folder will be created within code as: output_root_folder/MRC_output_folder
MRC_stat_results_folder = 'MRC_stat_results'
EPE_stat_results_folder = 'EPE_stat_results'
feature_storage_folder = 'features'
EPE_csv_file_name = 'results.csv'
coll_EPE_csv_file_name = 'coll_results.csv'
p_csv_path, auc_csv_path  = 'p_values.csv', 'auc_values.csv'
meaningful_stat_csv_path, selected_features_path = 'meaningful_stats.csv', 'selected_features.csv'
coll_p_csv_path, coll_auc_csv_path = 'p_values_collective.csv', 'auc_values_collective.csv'
coll_meaningful_stat_csv_path, coll_selected_features_path = 'meaningful_collective_stats.csv', 'selected_features_coll.csv'

## sequences info
sequence_dict = {'t1': ['t1_tra', 't1_sag']}

## analysis limits
AUC_limit = 0.7
p_limit = 0.01
minimum_step , maximum_step = 4, 5
minimum_power, maximum_power = 2, 3
primary_rate_calculation = 'all'
secondary_rate_calculation = 'all'
calculate_all_powers = True
calculate_all_steps = True
take_p_values_in_csv, take_auc_values_in_csv, take_meaningful_values_csv = True, True, True
EPE_analysis_proccess_follow = 'none'
EPE_specificity_min = 0.90
EPE_sensitivity_min = 0.60
EPE_analysis_per_case_research_value_limit = 20
processor_numbers_for_paralel_calculation = 4
processor_numbers_for_paralel_calculation_coll = 2

<a name='1'></a>
## 1. MRC analysis

In [10]:
MRC_Analysis = MRC_Analysis( stripped_data_folder,
    original_data_folder,
    output_root_folder,
    MRC_output_folder,
    MRC_analysis_features={'min_step': minimum_step, 'max_step': maximum_step, 
                            'min_power' :minimum_power, 'max_power' : maximum_power,
                           'ROI_shapes': ['circle', 'square'],
                           'primary_rate': primary_rate_calculation, 
                           'secondary_rate': secondary_rate_calculation, 
                           'calculate_all_powers': calculate_all_powers, 
                           'calculate_all_steps': calculate_all_steps})

MRC_Analysis.make_MRC_analysis()

0it [00:00, ?it/s]


<a name='2'></a>
## 2. MRC - Statistical analysis 

In [11]:
MRC_stat = MRC_statistical_analysis(stripped_data_folder = stripped_data_folder, 
                 output_root_folder = output_root_folder, 
                 MRC_output_folder = os.path.join(output_root_folder, MRC_output_folder),
                 stat_results_folder = MRC_stat_results_folder, 
              stat_limits = {'auc_limit' : AUC_limit, 'p_limit' : p_limit})

## make analysis per MRI type
MRC_stat.make_analysis(take_p_csv = take_p_values_in_csv, 
                       take_auc_csv = take_auc_values_in_csv, 
                       take_meaningful_csv = take_meaningful_values_csv)

MRC_stat.save_results(p_csv_path = p_csv_path, auc_csv_path = auc_csv_path, 
                      meaningful_stat_csv_path = meaningful_stat_csv_path,
                      selected_features_path = selected_features_path,
                     sort_columns = True)

## make analysis per sequence (collective analysis)
MRC_stat.make_collective_analysis(take_p_csv = take_p_values_in_csv, 
                                  take_auc_csv = take_auc_values_in_csv, 
                                  take_meaningful_csv = take_meaningful_values_csv,
                                 sequences = sequence_dict,
                                case_info_excel_file = os.path.join(output_root_folder, MRI_info_excel_name))

MRC_stat.save_results(p_csv_path = coll_p_csv_path, auc_csv_path = coll_auc_csv_path, 
                      meaningful_stat_csv_path = coll_meaningful_stat_csv_path,
                      selected_features_path = coll_selected_features_path,
                         sort_columns = True)

100%|█████████████████████████████████████████████| 1/1 [00:14<00:00, 14.21s/it]


<a name='3'></a>
## 3. EPE analysis 

In [ ]:
epe = EPE_Analysis__(stripped_data_folder = stripped_data_folder,
            original_data_folder = original_data_folder,
            output_root_folder = output_root_folder,
            stat_results = EPE_stat_results_folder, 
            feature_out = feature_storage_folder)

epe.make_analysis(features_csv_path = os.path.join(MRC_stat_results_folder, selected_features_path), 
                      EPE_stat_file_name = EPE_csv_file_name,
                      specificity_min = EPE_specificity_min, sensitivity_min = EPE_sensitivity_min, 
                      per_case_value_limit = EPE_analysis_per_case_research_value_limit,
                      record_features = True,
                      follow_proccess = EPE_analysis_proccess_follow,
                     processor_number = processor_numbers_for_paralel_calculation)

epe.make_collective_analysis(features_collective_csv_path = os.path.join(MRC_stat_results_folder, coll_selected_features_path), 
                      coll_EPE_stat_file_name = coll_EPE_csv_file_name,
                      sequences = sequence_dict,
                      case_info_excel_file = os.path.join(output_root_folder, MRI_info_excel_name),
                      specificity_min = EPE_specificity_min, sensitivity_min = EPE_sensitivity_min, 
                      per_case_value_limit = EPE_analysis_per_case_research_value_limit,
                      record_features = True,
                      follow_proccess = EPE_analysis_proccess_follow,
                     processor_number = processor_numbers_for_paralel_calculation_coll)